In [ ]:
! pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importing Libraries

In [ ]:
import pandas as pd
import wave
from pydub import AudioSegment
import os
from google.colab import drive
import collections
import random
from pathlib import Path
import numpy as np

## Mounting Google Drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Synthesize the Empty Wav Files for Slakh_2100

## Read the Instrument Frequency 

In [ ]:
slakh_inst_df = pd.read_csv('/content/drive/MyDrive/Mus-Comp/Collab Files/Source Separation Code Files/intrument_frequency.csv', keep_default_na=False)

In [ ]:
slakh_inst_df

,Mix,Acoustic Grand Piano,Bright Acoustic Piano,Electric Grand Piano,Honky-tonk Piano,Electric Piano 1,Electric Piano 2,Harpsichord,Clavi,Celesta,...,Guitar Fret Noise,Breath Noise,Seashore,Bird Tweet,Telephone Ring,Helicopter,Applause,Gunshot,Drums,Duration
0,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,,,,,,...,,,,,,,,,/Data_Set/MIDI Transcription/wav/slakh2100_000...,241.5571882
1,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,,,,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,...,,,,,,,,,/Data_Set/MIDI Transcription/wav/slakh2100_000...,190.3804082
2,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,,,,,,...,,,,,,,,,/Data_Set/MIDI Transcription/wav/slakh2100_000...,218.7203628
3,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,,,,,,...,,,,,,,,,,232.5942857
4,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,,,,,,...,,,,,,,,,,247.1531973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,Data_Set/MIDI Transcription/wav/slakh2100_0209...,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,,...,,,,,,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,176.4368254
2096,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,...,,,,,,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,299.1310658
2097,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,,,...,,,,,,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,78.61115646
2098,Data_Set/MIDI Transcription/wav/slakh2100_0209...,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,,,...,,,,,,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,206.6343764


## Dataframe Preprocessing 

In [ ]:
slakh_inst_df.columns

Index(['Mix', 'Acoustic Grand Piano', 'Bright Acoustic Piano',
       'Electric Grand Piano', 'Honky-tonk Piano', 'Electric Piano 1',
       'Electric Piano 2', 'Harpsichord', 'Clavi', 'Celesta',
       ...
       'Guitar Fret Noise', 'Breath Noise', 'Seashore', 'Bird Tweet',
       'Telephone Ring', 'Helicopter', 'Applause', 'Gunshot', 'Drums',
       'Duration'],
      dtype='object', length=131)

In [ ]:
instruments_needed = [ 
    'Electric Bass (pick)',
    'Glockenspiel',
    'Bright Acoustic Piano',
    'Rock Organ',
    'String Ensemble 1',
    'Distortion Guitar',
    'Brass Section' ,
    'Electric Grand Piano' ,
    'Electric Guitar (clean)' ,
    'Acoustic Guitar (nylon)',
    'Violin',
    'Drums']

In [ ]:
slakh_df_pruned = slakh_inst_df[['Mix'] + instruments_needed]
slakh_df_pruned.columns

Index(['Mix', 'Electric Bass (pick)', 'Glockenspiel', 'Bright Acoustic Piano',
       'Rock Organ', 'String Ensemble 1', 'Distortion Guitar', 'Brass Section',
       'Electric Grand Piano', 'Electric Guitar (clean)',
       'Acoustic Guitar (nylon)', 'Violin', 'Drums'],
      dtype='object')

In [ ]:
slakh_df_pruned

,Mix,Electric Bass (pick),Glockenspiel,Bright Acoustic Piano,Rock Organ,String Ensemble 1,Distortion Guitar,Brass Section,Electric Grand Piano,Electric Guitar (clean),Acoustic Guitar (nylon),Violin,Drums
0,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,,/Data_Set/MIDI Transcription/wav/slakh2100_000...
1,/Data_Set/MIDI Transcription/wav/slakh2100_000...,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,/Data_Set/MIDI Transcription/wav/slakh2100_000...
2,/Data_Set/MIDI Transcription/wav/slakh2100_000...,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,,,,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,/Data_Set/MIDI Transcription/wav/slakh2100_000...
3,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,,,,
4,/Data_Set/MIDI Transcription/wav/slakh2100_000...,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,,,,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...
2096,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,,,,,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,Data_Set/MIDI Transcription/wav/slakh2100_0209...
2097,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...
2098,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,,,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,Data_Set/MIDI Transcription/wav/slakh2100_0209...,,,Data_Set/MIDI Transcription/wav/slakh2100_0209...


### Change "MIDI Transcription" to "Transcription_transfer" in the file paths

In [ ]:
slakh_df_pruned.replace({'MIDI Transcription/wav': "Transcription_transfer"}, regex=True, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


### Renaming column names

In [ ]:
slakh_df_pruned.rename(columns={'Electric Bass (pick)': "Electric Bass pick", 'Electric Guitar (clean)': "Electric Guitar clean", 'Acoustic Guitar (nylon)': 'Acoustic Guitar nylon'}, inplace=True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


### We want to remove any files that do not actually exist (files that were originally "omitted" from slakh2100

In [ ]:
def check_and_replace_non_existing_files(df, current_path, columns):
  non_existing_paths = []
  for col in columns:
        for i, path in enumerate(df[col]):
          if path != '' and not Path(current_path + path).exists():
            non_existing_paths.append(path)
            # replace non-existing path with ''
            df.at[i, col] = ''
  return df, non_existing_paths

In [ ]:
print("There are", str(len(check_and_replace_non_existing_files(df=slakh_df_pruned, current_path="/content/drive/MyDrive/Mus-Comp/", columns=list(slakh_df_pruned.columns))[1])), "paths missing")

There are 1828 paths missing


In [ ]:
slakh_df_pruned = check_and_replace_non_existing_files(df=slakh_df_pruned, current_path="/content/drive/MyDrive/Mus-Comp/", columns=list(slakh_df_pruned.columns))[0]

In [ ]:
print("There are", str(len(check_and_replace_non_existing_files(df=slakh_df_pruned, current_path="/content/drive/MyDrive/Mus-Comp/", columns=list(slakh_df_pruned.columns))[1])), "paths missing")

There are 0 paths missing


### Make 'Mix' column as index

In [ ]:
slakh_df_pruned.set_index('Mix', inplace=True)

In [ ]:
slakh_df_pruned

,Electric Bass pick,Glockenspiel,Bright Acoustic Piano,Rock Organ,String Ensemble 1,Distortion Guitar,Brass Section,Electric Grand Piano,Electric Guitar clean,Acoustic Guitar nylon,Violin,Drums
Mix,,,,,,,,,,,,
/Data_Set/Transcription_transfer/slakh2100_00001_-1.wav,,,,,,/Data_Set/Transcription_transfer/slakh2100_000...,,,,,,/Data_Set/Transcription_transfer/slakh2100_000...
/Data_Set/Transcription_transfer/slakh2100_00002_-1.wav,/Data_Set/Transcription_transfer/slakh2100_000...,,,,,,/Data_Set/Transcription_transfer/slakh2100_000...,,,,,/Data_Set/Transcription_transfer/slakh2100_000...
/Data_Set/Transcription_transfer/slakh2100_00003_-1.wav,/Data_Set/Transcription_transfer/slakh2100_000...,,,,,,,,/Data_Set/Transcription_transfer/slakh2100_000...,,,/Data_Set/Transcription_transfer/slakh2100_000...
/Data_Set/Transcription_transfer/slakh2100_00004_-1.wav,,,,/Data_Set/Transcription_transfer/slakh2100_000...,,,,,,,,
/Data_Set/Transcription_transfer/slakh2100_00005_-1.wav,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
Data_Set/Transcription_transfer/slakh2100_02096_-1.wav,,,,,,,,,Data_Set/Transcription_transfer/slakh2100_0209...,,,Data_Set/Transcription_transfer/slakh2100_0209...
,,,,,,,,,,,,
Data_Set/Transcription_transfer/slakh2100_02098_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0209...,,,,,Data_Set/Transcription_transfer/slakh2100_0209...,,,,Data_Set/Transcription_transfer/slakh2100_0209...


### Check for duplicate paths across rows

In [ ]:
def check_dups_across_row(df):
  """This is for printing out rows which have duplicate column values
  """
  check_rows = []
  for row in df.itertuples():
    check_rows.append([item for item, count in collections.Counter(list(row)).items() if count > 1])
  for r in check_rows:
    if len(r) > 1:
      print(r)

In [ ]:
check_dups_across_row(slakh_df_pruned)

### We want to sort the `slakh_df_pruned` dataframe so that the rows with high number of instruments present appear at the top

In [ ]:
def count_non_empty_cells(df):
  """This is for counting cells in a dataframe that are NOT ''
  """
  non_empty_counts = []
  for row in df.itertuples():
    count = -1  # counter for number of full cells (start at -1 because it counts mix as a non-empty cell)
    for column_val in row:
      if column_val != '':  # if cell is not empty
        count += 1
    non_empty_counts.append(count)
  return non_empty_counts

In [ ]:
slakh_df_pruned['num_of_inst'] = count_non_empty_cells(df=slakh_df_pruned)
slakh_df_pruned

<ipython-input-71-d4be05a0dbf6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slakh_df_pruned['num_of_inst'] = count_non_empty_cells(df=slakh_df_pruned)


,Electric Bass pick,Glockenspiel,Bright Acoustic Piano,Rock Organ,String Ensemble 1,Distortion Guitar,Brass Section,Electric Grand Piano,Electric Guitar clean,Acoustic Guitar nylon,Violin,Drums,num_of_inst
Mix,,,,,,,,,,,,,
/Data_Set/Transcription_transfer/slakh2100_00001_-1.wav,,,,,,/Data_Set/Transcription_transfer/slakh2100_000...,,,,,,/Data_Set/Transcription_transfer/slakh2100_000...,2
/Data_Set/Transcription_transfer/slakh2100_00002_-1.wav,/Data_Set/Transcription_transfer/slakh2100_000...,,,,,,/Data_Set/Transcription_transfer/slakh2100_000...,,,,,/Data_Set/Transcription_transfer/slakh2100_000...,3
/Data_Set/Transcription_transfer/slakh2100_00003_-1.wav,/Data_Set/Transcription_transfer/slakh2100_000...,,,,,,,,/Data_Set/Transcription_transfer/slakh2100_000...,,,/Data_Set/Transcription_transfer/slakh2100_000...,3
/Data_Set/Transcription_transfer/slakh2100_00004_-1.wav,,,,/Data_Set/Transcription_transfer/slakh2100_000...,,,,,,,,,1
/Data_Set/Transcription_transfer/slakh2100_00005_-1.wav,,,,,,,,,,,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Data_Set/Transcription_transfer/slakh2100_02096_-1.wav,,,,,,,,,Data_Set/Transcription_transfer/slakh2100_0209...,,,Data_Set/Transcription_transfer/slakh2100_0209...,2
,,,,,,,,,,,,,-1
Data_Set/Transcription_transfer/slakh2100_02098_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0209...,,,,,Data_Set/Transcription_transfer/slakh2100_0209...,,,,Data_Set/Transcription_transfer/slakh2100_0209...,3


#### Sorting by `num_of_inst` in descending order so highest number comes at the top

In [ ]:
slakh_df_pruned.sort_values(by='num_of_inst', inplace=True, ascending=False)
slakh_df_pruned

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Electric Bass pick,Glockenspiel,Bright Acoustic Piano,Rock Organ,String Ensemble 1,Distortion Guitar,Brass Section,Electric Grand Piano,Electric Guitar clean,Acoustic Guitar nylon,Violin,Drums,num_of_inst
Mix,,,,,,,,,,,,,
Data_Set/Transcription_transfer/slakh2100_00028_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,,Data_Set/Transcription_transfer/slakh2100_0002...,8
Data_Set/Transcription_transfer/slakh2100_00656_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,,Data_Set/Transcription_transfer/slakh2100_0065...,8
Data_Set/Transcription_transfer/slakh2100_00029_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,,,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,7
Data_Set/Transcription_transfer/slakh2100_00100_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,,,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,7
Data_Set/Transcription_transfer/slakh2100_01590_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0159...,,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_transfer/slakh2100_0159...,,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_transfer/slakh2100_0159...,,Data_Set/Transcription_transfer/slakh2100_0159...,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,,,,,,,,,,,,,-1
,,,,,,,,,,,,,-1
,,,,,,,,,,,,,-1


### Splitting Training Set and Reserve Set

#### TRAIN, RESERVE SPLITTING PARAMETERS

In [ ]:
TRAIN_COUNT = 300

In [ ]:
training_wavs = slakh_df_pruned[:TRAIN_COUNT]
synth_reserve = slakh_df_pruned[TRAIN_COUNT:]

print("Training set contains", str(len(training_wavs)), "songs")
print("Reserve set contains", str(len(synth_reserve)), "songs")

Training set contains 300 songs
Reserve set contains 1800 songs


#### We need to remove songs from `synth_reserve` that have less than 2 instrument (so `num_of_inst` would be 0 or 1 or -1)

In [ ]:
synth_reserve = synth_reserve[synth_reserve.num_of_inst > 1]

In [ ]:
synth_reserve

,Electric Bass pick,Glockenspiel,Bright Acoustic Piano,Rock Organ,String Ensemble 1,Distortion Guitar,Brass Section,Electric Grand Piano,Electric Guitar clean,Acoustic Guitar nylon,Violin,Drums,num_of_inst
Mix,,,,,,,,,,,,,
Data_Set/Transcription_transfer/slakh2100_00819_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0081...,,Data_Set/Transcription_transfer/slakh2100_0081...,,Data_Set/Transcription_transfer/slakh2100_0081...,,,,,Data_Set/Transcription_transfer/slakh2100_0081...,4
Data_Set/Transcription_transfer/slakh2100_00627_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0062...,,Data_Set/Transcription_transfer/slakh2100_0062...,,Data_Set/Transcription_transfer/slakh2100_0062...,,,,,Data_Set/Transcription_transfer/slakh2100_0062...,4
Data_Set/Transcription_transfer/slakh2100_00597_-1.wav,Data_Set/Transcription_transfer/slakh2100_0059...,,,,,,Data_Set/Transcription_transfer/slakh2100_0059...,,,,Data_Set/Transcription_transfer/slakh2100_0059...,Data_Set/Transcription_transfer/slakh2100_0059...,4
Data_Set/Transcription_transfer/slakh2100_00629_-1.wav,,,,,Data_Set/Transcription_transfer/slakh2100_0062...,,,,Data_Set/Transcription_transfer/slakh2100_0062...,,Data_Set/Transcription_transfer/slakh2100_0062...,Data_Set/Transcription_transfer/slakh2100_0062...,4
Data_Set/Transcription_transfer/slakh2100_01604_-1.wav,Data_Set/Transcription_transfer/slakh2100_0160...,,,,Data_Set/Transcription_transfer/slakh2100_0160...,,Data_Set/Transcription_transfer/slakh2100_0160...,,,,,Data_Set/Transcription_transfer/slakh2100_0160...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Data_Set/Transcription_transfer/slakh2100_00023_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0002...,,,,,,,,,Data_Set/Transcription_transfer/slakh2100_0002...,2
Data_Set/Transcription_transfer/slakh2100_00072_-1.wav,,,,,Data_Set/Transcription_transfer/slakh2100_0007...,,,,,,,Data_Set/Transcription_transfer/slakh2100_0007...,2
Data_Set/Transcription_transfer/slakh2100_01094_-1.wav,,,,,Data_Set/Transcription_transfer/slakh2100_0109...,,,,,,,Data_Set/Transcription_transfer/slakh2100_0109...,2


## Addings paths to missing instruments in `training_wavs`

### Grabbing instrument paths from `synth_reserve` that can be added to missing cells in `training_wavs`

In [ ]:
source_sep_inst_dict = {
    'Electric Bass pick_path':[],
    'Glockenspiel_path':[],
    'Bright Acoustic Piano_path':[],
    'Rock Organ_path':[],
    'String Ensemble 1_path':[],
    'Distortion Guitar_path':[],
    'Brass Section_path' :[],
    'Electric Grand Piano_path' :[],
    'Electric Guitar clean_path' :[],
    'Acoustic Guitar nylon_path':[],
    'Violin_path':[],
    'Drums_path':[]
}

In [ ]:
def add_reserve_paths(inst_dict, reserve_df):
  for col in reserve_df.columns:
    for val in reserve_df[col].values:
        if val != '' and col != 'num_of_inst':
            print("Adding", val, "to", col + '_path')
            inst_dict[col+'_path'].append(val)

In [ ]:
add_reserve_paths(source_sep_inst_dict, synth_reserve)

Adding Data_Set/Transcription_transfer/slakh2100_00597_11.wav to Electric Bass pick_path
Adding Data_Set/Transcription_transfer/slakh2100_01604_05.wav to Electric Bass pick_path
Adding Data_Set/Transcription_transfer/slakh2100_00503_05.wav to Electric Bass pick_path
Adding Data_Set/Transcription_transfer/slakh2100_00483_13.wav to Electric Bass pick_path
Adding Data_Set/Transcription_transfer/slakh2100_00866_11.wav to Electric Bass pick_path
Adding Data_Set/Transcription_transfer/slakh2100_00670_04.wav to Electric Bass pick_path
Adding Data_Set/Transcription_transfer/slakh2100_01586_00.wav to Electric Bass pick_path
Adding Data_Set/Transcription_transfer/slakh2100_00664_04.wav to Electric Bass pick_path
Adding Data_Set/Transcription_transfer/slakh2100_01580_06.wav to Electric Bass pick_path
Adding Data_Set/Transcription_transfer/slakh2100_01398_05.wav to Electric Bass pick_path
Adding Data_Set/Transcription_transfer/slakh2100_02052_11.wav to Electric Bass pick_path
Adding Data_Set/Trans

In [ ]:
for key, value in source_sep_inst_dict.items():
  print("Reserve:", key, len(value))  # show the value counts for each instrument

Reserve: Electric Bass pick_path 58
Reserve: Glockenspiel_path 25
Reserve: Bright Acoustic Piano_path 430
Reserve: Rock Organ_path 123
Reserve: String Ensemble 1_path 493
Reserve: Distortion Guitar_path 228
Reserve: Brass Section_path 166
Reserve: Electric Grand Piano_path 88
Reserve: Electric Guitar clean_path 437
Reserve: Acoustic Guitar nylon_path 205
Reserve: Violin_path 39
Reserve: Drums_path 1215


#### Finding out the number of missing files for each instrument in `training_wavs`

In [ ]:
for key in source_sep_inst_dict:
  inst = key.split("_")[0]
  miss_count = len(training_wavs[training_wavs[inst] == ''])
  present_count = len(source_sep_inst_dict[key])
  print(inst,  str(miss_count), "out of", str(TRAIN_COUNT), "are missing,", str(present_count), "to choose from,", str(TRAIN_COUNT - miss_count), "present")

Electric Bass pick 262 out of 300 are missing, 58 to choose from, 38 present
Glockenspiel 285 out of 300 are missing, 25 to choose from, 15 present
Bright Acoustic Piano 126 out of 300 are missing, 430 to choose from, 174 present
Rock Organ 213 out of 300 are missing, 123 to choose from, 87 present
String Ensemble 1 109 out of 300 are missing, 493 to choose from, 191 present
Distortion Guitar 172 out of 300 are missing, 228 to choose from, 128 present
Brass Section 205 out of 300 are missing, 166 to choose from, 95 present
Electric Grand Piano 238 out of 300 are missing, 88 to choose from, 62 present
Electric Guitar clean 108 out of 300 are missing, 437 to choose from, 192 present
Acoustic Guitar nylon 211 out of 300 are missing, 205 to choose from, 89 present
Violin 269 out of 300 are missing, 39 to choose from, 31 present
Drums 0 out of 300 are missing, 1215 to choose from, 300 present


### Merging random **reserve** instrument *wavs* to **mix** *wavs* in `training_wavs` and adding their respective paths in missing cells 

In [ ]:
def merge_mix(mix_wav_file_path, inst_wav_file_path, inst_destination_path, mix_destination_path):
  # Open the mix file and read its audio data
  mix = AudioSegment.from_file(mix_wav_file_path, format="wav")

  # Open the single instrument file and read its audio data
  single_inst = AudioSegment.from_file(inst_wav_file_path, format="wav")

  # Check if single_inst is shorter than mix
  if single_inst.duration_seconds < mix.duration_seconds:
      # Calculate the number of times single_inst needs to be repeated
      num_repeats = int(mix.duration_seconds / single_inst.duration_seconds) + 1

      # Concatenate single_inst with itself the necessary number of times
      repeated_single_inst = single_inst * num_repeats

      # Truncate repeated_single_inst to the duration of mix
      repeated_single_inst = repeated_single_inst[:mix.duration_seconds * 1000]

      # Export the modified single instrument audio data to a new WAV file
      repeated_single_inst.export(inst_destination_path, format="wav")

      # Overlay the repeated_single_inst audio data on top of the mix audio data
      overlayed_audio = mix.overlay(repeated_single_inst)
  else:
    # Overlay the single instrument audio data on top of the mix audio data
    overlayed_audio = mix.overlay(single_inst)

    # Make single_inst the same duration as mix
    single_inst = single_inst[:mix.duration_seconds * 1000]

    # Export the modified single instrument audio data to a new WAV file
    single_inst.export(inst_destination_path, format="wav")

  # Export the overlayed audio data to a new WAV file
  overlayed_audio.export(mix_destination_path, format="wav")

#### TEST: MERGING TWO **INSTRUMENTS** together to see if the function above works

We treat one instrument as "MIX"

In [ ]:
start_path = "/content/drive/MyDrive/Mus-Comp/"

first_row = slakh_df_pruned.iloc[0]  # Duration is 4:15
second_row = slakh_df_pruned.iloc[2]  # Duration is 2:01, SO TREAT THIS ONE AS INSTRUMENT FILE

mix_wav_fp = start_path + first_row['Bright Acoustic Piano']
inst_wav_fp = start_path + second_row['Drums']

print("The Mix wav file is:", mix_wav_fp)
print("The Instrument wav file is:", inst_wav_fp)

merge_mix(mix_wav_fp, inst_wav_fp, inst_destination_path="modified_single_inst.wav", mix_destination_path="overlayed.wav")  # Should be 4:15 in duration with Piano and Drums playing togther

The Mix wav file is: /content/drive/MyDrive/Mus-Comp/Data_Set/Transcription_transfer/slakh2100_00028_00.wav
The Instrument wav file is: /content/drive/MyDrive/Mus-Comp/Data_Set/Transcription_transfer/slakh2100_00029_08.wav


In [ ]:
start_path = "/content/drive/MyDrive/Mus-Comp/"

first_row = slakh_df_pruned.iloc[0]  # Duration is 4:15, SO TREAT THIS ONE AS INSTRUMENT FILE
second_row = slakh_df_pruned.iloc[2]  # Duration is 2:01

mix_wav_fp = start_path + second_row['Drums']
inst_wav_fp = start_path + first_row['Bright Acoustic Piano']

print("The Mix wav file is:", mix_wav_fp)
print("The Instrument wav file is:", inst_wav_fp)

merge_mix(mix_wav_fp, inst_wav_fp, inst_destination_path="modified_single_inst.wav", mix_destination_path="overlayed.wav")  # Should be 2:01 in duration with Piano and Drums playing togther

The Mix wav file is: /content/drive/MyDrive/Mus-Comp/Data_Set/Transcription_transfer/slakh2100_00029_08.wav
The Instrument wav file is: /content/drive/MyDrive/Mus-Comp/Data_Set/Transcription_transfer/slakh2100_00028_00.wav


### Now we need to merge missing instrument files into the mixes in `training_wavs` and add their respective paths in the DataFrame

In [ ]:
training_wavs

,Electric Bass pick,Glockenspiel,Bright Acoustic Piano,Rock Organ,String Ensemble 1,Distortion Guitar,Brass Section,Electric Grand Piano,Electric Guitar clean,Acoustic Guitar nylon,Violin,Drums,num_of_inst
Mix,,,,,,,,,,,,,
Data_Set/Transcription_transfer/slakh2100_00028_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,,Data_Set/Transcription_transfer/slakh2100_0002...,8
Data_Set/Transcription_transfer/slakh2100_00656_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,,Data_Set/Transcription_transfer/slakh2100_0065...,8
Data_Set/Transcription_transfer/slakh2100_00029_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,,,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,7
Data_Set/Transcription_transfer/slakh2100_00100_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,,,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,7
Data_Set/Transcription_transfer/slakh2100_01590_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0159...,,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_transfer/slakh2100_0159...,,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_transfer/slakh2100_0159...,,Data_Set/Transcription_transfer/slakh2100_0159...,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Data_Set/Transcription_transfer/slakh2100_01254_-1.wav,Data_Set/Transcription_transfer/slakh2100_0125...,,,,Data_Set/Transcription_transfer/slakh2100_0125...,,,Data_Set/Transcription_transfer/slakh2100_0125...,,,,Data_Set/Transcription_transfer/slakh2100_0125...,4
Data_Set/Transcription_transfer/slakh2100_00815_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0081...,,,,Data_Set/Transcription_transfer/slakh2100_0081...,,,Data_Set/Transcription_transfer/slakh2100_0081...,,Data_Set/Transcription_transfer/slakh2100_0081...,4
Data_Set/Transcription_transfer/slakh2100_01605_-1.wav,,,Data_Set/Transcription_transfer/slakh2100_0160...,Data_Set/Transcription_transfer/slakh2100_0160...,,,,,Data_Set/Transcription_transfer/slakh2100_0160...,,,Data_Set/Transcription_transfer/slakh2100_0160...,4


In [ ]:
def get_random_song_from_reserve(instrument):
  instrument_path_str = instrument + "_path"
  return random.choice(source_sep_inst_dict[instrument_path_str])

In [ ]:
get_random_song_from_reserve("Electric Bass pick")

'Data_Set/Transcription_transfer/slakh2100_00172_07.wav'

In [ ]:
def merge_and_fill_empty_training_wavs(df, merge_and_single_inst_path_folder, current_start_path):
  # create boolean mask indicating which cells contain empty strings
  mask = df.isin([''])

  # get row and column indices of cells that meet the condition
  rows, cols = np.where(mask)

  # get column names corresponding to column numbers
  col_names = df.columns

  for row, col in zip(rows, cols):
      col_name = col_names[col]
      print("Missing cell at", str(row), str(col), "for", col_name)
      random_inst_path_chosen = get_random_song_from_reserve(col_name)
      print("Random", col_name, "wav file picked from reserve:", random_inst_path_chosen)
      print("Merging the mix wav", df.index[row], "with", random_inst_path_chosen)
      mix_path = current_start_path + df.index[row]
      inst_path = current_start_path + random_inst_path_chosen
      print('DFINDEX',df.index[row])
      track_path = '_'.join(df.index[row].split('/')[2].split('_')[:2])
      inst_dest_path = current_start_path + merge_and_single_inst_path_folder + track_path + "_" + col_name + ".wav"
      mix_dest_path = current_start_path + merge_and_single_inst_path_folder + track_path + "_" + "-1.wav"
      merge_mix(mix_wav_file_path=mix_path, inst_wav_file_path=inst_path, inst_destination_path=inst_dest_path, mix_destination_path=mix_dest_path)
      print("Merging done. New mix destination:", mix_dest_path, "\nNew instrument destination:", inst_dest_path + "\n")
      df.iloc[row, col] = merge_and_single_inst_path_folder + track_path + "_" + col_name + ".wav"
      df.index.values[row] = merge_and_single_inst_path_folder + track_path + "_" + "-1.wav"


In [ ]:
training_wavs_copy = training_wavs.copy()
training_wavs_copy = training_wavs_copy.drop('', axis=0)
merge_and_fill_empty_training_wavs(training_wavs_copy, merge_and_single_inst_path_folder="Data_Set/Transcription_Destination/", current_start_path=start_path)

Streaming output truncated to the last 5000 lines.
New instrument destination: /content/drive/MyDrive/Mus-Comp/Data_Set/Transcription_Destination/slakh2100_01652_Electric Grand Piano.wav

Missing cell at 209 8 for Electric Guitar clean
Random Electric Guitar clean wav file picked from reserve: Data_Set/Transcription_transfer/slakh2100_00654_02.wav
Merging the mix wav Data_Set/Transcription_Destination/slakh2100_01652_-1.wav with Data_Set/Transcription_transfer/slakh2100_00654_02.wav
DFINDEX Data_Set/Transcription_Destination/slakh2100_01652_-1.wav
Merging done. New mix destination: /content/drive/MyDrive/Mus-Comp/Data_Set/Transcription_Destination/slakh2100_01652_-1.wav 
New instrument destination: /content/drive/MyDrive/Mus-Comp/Data_Set/Transcription_Destination/slakh2100_01652_Electric Guitar clean.wav

Missing cell at 209 10 for Violin
Random Violin wav file picked from reserve: Data_Set/Transcription_transfer/slakh2100_01998_05.wav
Merging the mix wav Data_Set/Transcription_Destin

In [ ]:
training_wavs_copy

,Electric Bass pick,Glockenspiel,Bright Acoustic Piano,Rock Organ,String Ensemble 1,Distortion Guitar,Brass Section,Electric Grand Piano,Electric Guitar clean,Acoustic Guitar nylon,Violin,Drums,num_of_inst
Mix,,,,,,,,,,,,,
Data_Set/Transcription_Destination/slakh2100_00028_-1.wav,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0002...,8
Data_Set/Transcription_Destination/slakh2100_00656_-1.wav,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_transfer/slakh2100_0065...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0065...,8
Data_Set/Transcription_Destination/slakh2100_00029_-1.wav,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0002...,Data_Set/Transcription_transfer/slakh2100_0002...,7
Data_Set/Transcription_Destination/slakh2100_00100_-1.wav,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0010...,Data_Set/Transcription_transfer/slakh2100_0010...,7
Data_Set/Transcription_Destination/slakh2100_01590_-1.wav,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_transfer/slakh2100_0159...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0159...,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Data_Set/Transcription_Destination/slakh2100_01254_-1.wav,Data_Set/Transcription_transfer/slakh2100_0125...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_transfer/slakh2100_0125...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Transcription_Destination/slakh2100_0...,Data_Set/Tran

In [ ]:
training_wavs_copy.to_csv('training.csv')